[View in Colaboratory](https://colab.research.google.com/github/orico/UnitTestAndDecoratorLogger/blob/master/Q1_final.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/Colab Notebooks/Interviewhomeworktask 2018/")
!ls


In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import os
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier  


%matplotlib inline

# !pip install gensim
# import gensim
# !pip install graphviz
# import graphviz
!ls


interviewClassificationTask.csv  Q2.ipynb			   tree.dot
Q1.ipynb			 Source.gv			   unigrams
Q2 - Final.ipynb		 StuccomediaInterviewHomework.pdf


The first question is a pure data science task and we would like to see a python notebook with the steps.

Question 1:

Attached are short text and their classification to either 0\1. Write a classifier in python that

predicts the class (python notebook that describes the steps towards solution)
The file attached.

In [3]:
df = pd.read_csv('interviewClassificationTask.csv',encoding = "ISO-8859-1")
df = df.drop(df.columns[[2, 3, 4]], axis=1)

print('unbalanced classes:')
print('class 0:', (float(df.shape[0]) - float(np.sum(df['v1']))) / float(df.shape[0]) * 100) 
print('class 1:', float(np.sum(df['v1'])) / float(df.shape[0]) * 100)

unbalanced classes:
class 0: 86.59368269921033
class 1: 13.406317300789663


In [4]:
nltk.download('stopwords')
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)
corpus = df['v2'].values
norm_corpus = normalize_corpus(corpus)
print(norm_corpus)

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['go jurong point crazy available bugis n great world la e buffet cine got amore wat'
 'ok lar joking wif u oni'
 'free entry wkly comp win fa cup final tkts st may text fa receive entry questionstd txt ratetcs apply overs'
 ... 'pity mood soany suggestions'
 'guy bitching acted like id interested buying something else next week gave us free'
 'rofl true name']


In [5]:
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
print(cv_matrix)
print(cv_matrix.shape)

# # get all unique words in the corpus
# vocab = cv.get_feature_names()
# # show document feature vectors
# p = pd.DataFrame(cv_matrix, columns=vocab)
# print(p.shape)
# print(p.head())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(5572, 8389)


In [6]:
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)
print(bv_matrix.shape)
bv_matrix = bv_matrix.toarray()
print(bv_matrix.shape)

# vocab = bv.get_feature_names()
# pd.DataFrame(bv_matrix, columns=vocab)

(5572, 30433)
(5572, 30433)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
print(tv_matrix.shape)
tv_matrix = tv_matrix.toarray()
print(tv_matrix.shape)
# vocab = tv.get_feature_names()
# pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

(5572, 8389)
(5572, 8389)


In [10]:
def train_test(matrix):
  X = matrix
  y = df['v1']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  print(X_train.shape)
  print(X_test.shape)

  print('\nlabel ratio in train-set:')
  print('class 0:', (float(X_train.shape[0]) - float(np.sum(y_train))) / float(X_train.shape[0]) * 100) 
  print('class 1:', float(np.sum(y_train)) / float(X_train.shape[0]) * 100)
  print()
  
  print('\nlabel ratio in test-set:')
  print('class 0:', (float(X_test.shape[0]) - float(np.sum(y_test))) / float(X_test.shape[0]) * 100) 
  print('class 1:', float(np.sum(y_test)) / float(X_test.shape[0]) * 100)
  print()
  
  clf = tree.DecisionTreeClassifier()
  clf.fit(X, y)
  prediction = clf.predict(X_test)
  print('Decision Tree:')
  print('\n', confusion_matrix(y_test, prediction))
  print('\n', 'Train/Test Accuracy: %', clf.score(X_test, y_test) * 100 )  
  tree.export_graphviz(clf, out_file='tree.dot')  
  print()
  
  
  clf2 = RandomForestClassifier(max_depth=50, random_state=0)
  clf2.fit(X, y)
  prediction = clf2.predict(X_test)
  print('Random Forest:')
  print('\n', confusion_matrix(y_test, prediction))
  print('\n', 'Train/Test Accuracy: %', clf2.score(X_test, y_test) * 100 )  
  print()
  
  clf3 = GaussianNB()
  clf3.fit(X, y)
  prediction = clf3.predict(X_test)
  print('GaussianNB:')
  print('\n', confusion_matrix(y_test, prediction))
  print('\n', 'Train/Test Accuracy: %', clf3.score(X_test, y_test) * 100 )  
  print()
  return clf


def KFold_split(matrix):  
  X = matrix
  y = df['v1']
  
  cv = KFold(n_splits = 3, shuffle = True)
  clf = tree.DecisionTreeClassifier()
  scores = cross_val_score(clf, X, y, cv=cv)
  print('Decision Tree CV scores:', scores, '\nmean:', np.mean(scores))

clf = train_test(cv_matrix)
KFold_split(cv_matrix)   

# # train_test(bv_matrix) 
# # KFold_split(bv_matrix)  

# KFold_split(tv_matrix)  
# train_test(tv_matrix) 

# uni_bi = np.hstack((cv_matrix,bv_matrix))
# train_test(uni_bi) 
# KFold_split(uni_bi) 

# unibitf = np.hstack((uni_bi,tv_matrix))
# uni_bi = ''  
# cv_matrix = ''  
# bv_matrix = ''  
# tv_matrix = ''  
# train_test(unibitf)
# KFold_split(unibitf) 

(4179, 8389)
(1393, 8389)

label ratio in train-set:
class 0: 86.69538167025604
class 1: 13.304618329743958


label ratio in test-set:
class 0: 86.28858578607323
class 1: 13.711414213926776

Decision Tree:

 [[1202    0]
 [   0  191]]

 Train/Test Accuracy: % 100.0

Random Forest:

 [[1202    0]
 [  27  164]]

 Train/Test Accuracy: % 98.06173725771716

GaussianNB:

 [[1107   95]
 [   0  191]]

 Train/Test Accuracy: % 93.18018664752333

Decision Tree CV scores: [0.96770721 0.9628433  0.95799677] 
mean: 0.9628490922254432
